In [1]:
import pandas as pd
from arango import ArangoClient

In [2]:
client = ArangoClient(hosts="https://2ae4f052d710.arangodb.cloud:8529")

db = client.db('machine_learning', username='lab_test', password='z-rRdN-Enf4qQwybGiVdbG')

In [3]:
pregel = db.pregel

In [59]:
job_id = db.pregel.create_job(
    graph='organization_organization_transaction',
    algorithm='effectivecloseness',
    store=False,
    max_gss=None,
    thread_count=1,
    async_mode=False,
    result_field='score'
)

In [60]:
job = pregel.job(job_id)

In [66]:
aql_query = """
FOR res IN PREGEL_RESULT(@job_id)
    RETURN {"_key": res._key, "effective_closeness": ROUND(res.score*100)/100}
"""
cursor = db.aql.execute(aql_query, bind_vars={'job_id': job_id})
scores = list(cursor)

In [57]:
if db.has_collection('organization_organization_transaction_closeness'):
    collection = db.collection('organization_organization_transaction_closeness')
else:
    collection = db.create_collection('organization_organization_transaction_closeness')

In [68]:
collection.insert_many(scores, overwrite_mode="replace", silent=True)

True

In [69]:
collection.count()

2072